## **3. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données
- Étiquetage morphosyntaxique (POS Tagging) 
- Lemmatisation
- Chunking / Filtrage par patrons syntaxiques (basés sur les termes MeSH)


**NLTK**\
https://www.nltk.org/ 

In [ ]:
import nltk
#nltk.download(['popular'])

In [ ]:
path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
acteur = 'cisss_ciusss'

In [ ]:
with open(path + acteur + '/' + acteur + '_corpus.txt', 'r', encoding='utf-8') as f:
    corpus = ''.join([x for x in f.read().lower() if x.isprintable()])

In [ ]:
ech = corpus[:round(len(corpus)/10)]

### **Segmentation** (phrases)

In [ ]:
from nltk import sent_tokenize 

sents = [s.strip('.') for s in sent_tokenize(ech)]

### **Tokenisation** (mots)

In [ ]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w+")

In [ ]:
tokens = [tokenizer_re.tokenize(s) for s in sents]

### **Filtrage** (antidictionnaire)

In [ ]:
# Importer l'antidictionnaire pour filtrer les données
from pandas import *

# Stopwords fréquents en français
path = "/Users/camilledemers/Documents/04-filtrage/stopwords.csv"
with open(path, 'r', encoding="utf-8") as f:
    stopwords = read_csv(f)
    stopwords = [t.lower() for t in stopwords['Stopwords'].tolist()]


# Stopwords fréquents en anglais
path = '/Users/camilledemers/Documents/04-filtrage/stop_words_english.txt'
with open(path, 'r', encoding="utf-8") as f:
    sw = [w.strip('\n').lower() for w in f.readlines()]

stopwords += sw

# Signes de ponctuation
import string 
punct = [s for s in string.punctuation] 
punct += ['»' ,'©', '']

stopwords += punct

#Prénoms (curieusement, il y en a beaucoup dans les données)

# Mis en commentaire pour l'instant car ça allonge le délai de traitement

# path = '/Users/camilledemers/Documents/04-filtrage/Prenoms.csv'
# with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['01_prenom'].tolist()]

# stopwords += sw

#Noms de famille 
# path = '/Users/camilledemers/Documents/04-filtrage/nomsFamille.csv'
# with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['Nom'].tolist()]

# stopwords += sw


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/mwe_stopwords.txt'

with open (path, 'r', encoding='utf-8') as f:
    mwe_sw = [tuple(tokenizer_re.tokenize(t)) for t in f.readlines()]
    #mwe_sw = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
from nltk.tokenize import MWETokenizer
tokenizer_mwe = MWETokenizer(mwe_sw, separator=' ')

mwe_sw = [tokenizer_mwe.tokenize(w)[0] for w in mwe_sw]

In [ ]:
tokens = [[t for t in tokenizer_mwe.tokenize(sent) if t not in mwe_sw] for sent in tokens]

### **Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook

In [ ]:
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.probability import FreqDist

In [ ]:
# Créer une fonction pour importer en CSV un tableau de termes triés par distribution de fréquences
import pandas as pd

def tabCSV (tab, titre):
    path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
    tab = pd.DataFrame(list(tab.items()), columns= ["Terme", "Fréquence"])
    tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
    tab.to_csv(path + titre + '.csv')

In [ ]:
from nltk.util import flatten, everygrams
ngrammes = [x for xs in [list(everygrams(sent, min_len=2, max_len=10)) for sent in tokens] for x in xs]

In [ ]:
# On retire maintenant les n-grammes qui débutent ou terminent par :
# - un stopword
# - un chiffre
# - un mot de deux lettres ou moins
def filtreNgrams(ngramme):
    return [" ".join(t) for t in ngramme if not t[0] in stopwords and not t[0].isnumeric() and not t[-1] in stopwords and not t[-1].isnumeric() and len(t[0]) > 2 and len(t[-1]) > 2] 

terms = filtreNgrams(ngrammes)

In [ ]:
freq = FreqDist(terms)
tabCSV(freq, acteur + '_n-grams')

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières 

kw = ['programme', 'service', 'intervenant', 'institut', 'groupe de recherche']

In [ ]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', "Fréquence"])
kwic = {w : [] for w in kw} 

In [ ]:
for t in terms:
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])

path = '/Users/camilledemers/Documents/04-filtrage' + '/' + acteur + '/'
extrant.to_csv(path + acteur + '_KWIC' +'.csv')

In [ ]:
# Pour la suite du traitement, on ne retient que les N expressions les plus fréquentes dans le corpus (ex. 5000)
#terms = [t[0] for t in list(freq.most_common(10000))] --> ça ne fonctionne pas ensuite pour compter les nb de fois qu'un terme MeSH se trouve dans le corpus
# OUI : voir cours classification LNG3120

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh-fr.txt'

with open (path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize(terms)

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)



In [ ]:
termes_mesh = FreqDist(termes_mesh)
tabCSV(termes_mesh, acteur + '_MeSH')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/SNOMED/SNOMED_fr.csv'

with open(path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize(terms)

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
termes_sm = FreqDist(termes_sm)
tabCSV(termes_sm, acteur + '_SNOMED')

### **POS Tagging**
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [ ]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

In [ ]:
tagged = [tagger.tag_text(term) for term in terms]

### **Lemmatisation**

In [ ]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [ ]:
tuples = []
tuples_lemmes = []
for terme in tagged:
    exp = " ".join([(t.split("\t")[0]) for t in terme])
    lemme = " ".join([lemmatizer.lemmatize(t) for t in terme])
    chunk = " ".join([(t.split("\t")[1]) for t in terme])
    tuples.append([exp, chunk])
    tuples_lemmes.append([lemme, chunk])

### **Filtrage - Patrons syntaxiques**

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:50] # On prend les 50 structures syntaxiques les plus fréquentes dans les MeSH

In [ ]:
tuples_lemmes = [t for t in tuples_lemmes if t[1] in patterns]

In [ ]:
tuples = [t for t in tuples if t[1] in patterns]

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
tab = pd.DataFrame(tuples, columns= ["Expression", "Structure syntaxique"])
tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
tab.to_csv(path + acteur + '_phrases.csv')

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
tab = pd.DataFrame(tuples_lemmes, columns= ["Expression", "Structure syntaxique"])
tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
tab.to_csv(path + acteur + '_phrases_lemmatized.csv')